In [362]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
from statsmodels.tools.eval_measures import rmse
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

sample_output = pd.read_csv('/Users/liaoshousan/Desktop/Career Hackthon/AOI/AOI Output Sample.csv')
sample_output

,Item,Defect
0,1,1
1,2,0
2,3,1
3,4,0
4,5,1
5,6,0
6,7,1
7,8,0
8,9,0
9,10,1


In [363]:

resize_size = 128
total_img = 10
X = np.zeros((total_img,resize_size,resize_size,1),dtype=np.uint8) 
y = sample_output['Defect']
from keras.utils.np_utils import *

Y = to_categorical(y, 2)

for num in range(total_img):
    if num+1!=total_img:
        file_name = '/Users/liaoshousan/Desktop/Career Hackthon/AOI/sample/00'+ str(num+1) +'.jpg'
    else:
        file_name = '/Users/liaoshousan/Desktop/Career Hackthon/AOI/sample/0'+ str(num+1) +'.jpg'


    img = Image.open(file_name).convert('L')
    
    new_img = img.resize((resize_size,resize_size),Image.LANCZOS)
    arr = np.array(new_img)
    X[num,:,:,:] = arr.reshape(128,128,1)
    

In [364]:
# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.5, random_state=5)


In [365]:
x_train = X[:6,:,:,:]
y_train = y[:6]
x_test = X[6:,:,:,:]
y_test = y[6:]

# # the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]


In [366]:
x_train.shape

(6, 128, 128, 1)

In [367]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

num_classes = 2
epochs = 20


In [368]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# input_shape = x_train.shape[1:]


In [369]:
from keras.preprocessing.image import ImageDataGenerator

import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras.regularizers import l2

import os
%matplotlib inline



def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)


def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    return Model(input, x)

def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))



In [370]:
# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(x_test, digit_indices)

# network definition
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)


In [371]:
print(base_network.summary())

Model: "model_59"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_116 (InputLayer)       (None, 128, 128, 1)       0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 16384)             0         
_________________________________________________________________
dense_95 (Dense)             (None, 128)               2097280   
_________________________________________________________________
dropout_57 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_96 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_58 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_97 (Dense)             (None, 128)               165

In [372]:
te_pairs.shape

(4, 2, 128, 128, 1)

In [378]:
# train
rms = RMSprop()
model.compile(loss='binary_crossentropy', optimizer=rms, metrics=[accuracy])
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          batch_size=128,
          epochs=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y)) #這裡有問題

# compute final accuracy on training and test sets
y_pred1 = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred1)
y_pred2 = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred2)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

Train on 8 samples, validate on 4 samples
Epoch 1/20
8/8 [==============================] - 1s 123ms/step - loss: 1.0473 - accuracy: 0.7500 - val_loss: 7.9712 - val_accuracy: 0.5000
Epoch 2/20
8/8 [==============================] - 0s 9ms/step - loss: 7.9712 - accuracy: 0.5000 - val_loss: 7.9712 - val_accuracy: 0.5000
Epoch 3/20
8/8 [==============================] - 0s 7ms/step - loss: 7.9712 - accuracy: 0.5000 - val_loss: 7.9712 - val_accuracy: 0.5000
Epoch 4/20
8/8 [==============================] - 0s 7ms/step - loss: 7.9712 - accuracy: 0.5000 - val_loss: 7.9712 - val_accuracy: 0.5000
Epoch 5/20
8/8 [==============================] - 0s 6ms/step - loss: 7.9712 - accuracy: 0.5000 - val_loss: 7.9712 - val_accuracy: 0.5000
Epoch 6/20
8/8 [==============================] - 0s 7ms/step - loss: 7.9712 - accuracy: 0.5000 - val_loss: 7.9712 - val_accuracy: 0.5000
Epoch 7/20
8/8 [==============================] - 0s 8ms/step - loss: 7.9712 - accuracy: 0.5000 - val_loss: 7.9712 - val_accurac

In [379]:
y_pred1

array([[1.5933415 ],
       [2.349592  ],
       [2.0454595 ],
       [0.48737046],
       [0.7837834 ],
       [2.349592  ],
       [1.1778032 ],
       [0.48737046]], dtype=float32)

In [380]:
y_train

0    1
1    0
2    1
3    0
4    1
5    0
Name: Defect, dtype: int64